<a href="https://colab.research.google.com/github/estefanabermeo/RegresionNoLineal_RegresionLogistica/blob/main/A7_2_Gastos2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ACTIVIDAD 7.2 REGRESIÓN LOGÍSTICA - GASTOS 2O22**
EQUIPO 10:
* Estefana Bermeo Severiano - A01367558
* Miguel Saúl Fernández Ávalos - A01707491
* Mariel Quetzali Fernández Montes - A0170779
* Christian Jesús Soto-Vieyra Gil - A01707759

---

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Leemos el archivo ya limpio
gastos22 = pd.read_csv("gastos2022_limpio.csv")
gastos22.head(5)

,Unnamed: 0,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,TC,Importe,IVA,RET ISR,RET IVA,Otros,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,0,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,1.0,5551.38,888.22,0.0,0.0,0.0,6439.6,6439.6,I,Vigente,PE02-ENE22
1,1,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,1.0,5551.98,888.32,0.0,0.0,0.0,6440.3,6440.3,I,Vigente,PE03-ENE22
2,2,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,Servicios de Facturación,PUE,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,I,Vigente,--
3,3,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,--,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,P,Vigente,--
4,4,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,--,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,P,Vigente,--


In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(gastos22['TIPO GASTO'])
unico

array(['ALIMENTOS', 'ANTICIPO ', 'ANTICIPO COMPRA AUTO',
       'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL', 'CALIBRACIONES',
       'CAPACITACION', 'CASETAS', 'COMISION BANCARIA', 'COMISION MIXTA',
       'COMPRA AUTO ', 'COMPRAS', 'COMPRAS ', 'CREDITO ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'GASOLINA', 'GASTOS IMPORTACION ',
       'GERENCIA ADMON', 'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT',
       'INSTALACION ELECTRICA', 'MAQUILAS', 'MAQUINARIA', 'MEDICOS',
       'MENSAJERIA', 'MONITOREO CAMARAS ADT', 'MTTO COMPUTO',
       'MTTO EQUIPO COMPUTO', 'MTTO LOCAL', 'MTTO MAQUINARIA',
       'MTTO OFICINA ', 'MTTO TRANSPORTE', 'NO DEDUCIBLE', 'PAPELERIA',
       'PAQUETERIA', 'PEAJES', 'PUBLICIDAD', 'RENTA AUTO',
       'RENTA COPIADORA', 'SANITIZACION', 'SEGUROS', 'SUBCONTRATOS',
       'SUELDOS DIRECCION', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORMES', 'VALES DESPENSA', 'VARIOS'], dtype=objec

Convertir la variable TIPO GASTO en una variable dicotómica

In [ ]:
#Convierto una variable a dicotómica 
gastos22['TIPO GASTO']= gastos22['TIPO GASTO'].replace(['COMPRAS '], "COMPRAS")

In [ ]:
#Convierto una variable a dicotómica 
gastos22['TIPO GASTO']= gastos22['TIPO GASTO'].replace(['ALIMENTOS', 'ANTICIPO ', 'ANTICIPO COMPRA AUTO',
       'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL', 'CALIBRACIONES',
       'CAPACITACION', 'CASETAS', 'COMISION BANCARIA', 'COMISION MIXTA',
       'COMPRA AUTO ', 'CREDITO ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'GASOLINA', 'GASTOS IMPORTACION ',
       'GERENCIA ADMON', 'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT',
       'INSTALACION ELECTRICA', 'MAQUILAS', 'MAQUINARIA', 'MEDICOS',
       'MENSAJERIA', 'MONITOREO CAMARAS ADT', 'MTTO COMPUTO',
       'MTTO EQUIPO COMPUTO', 'MTTO LOCAL', 'MTTO MAQUINARIA',
       'MTTO OFICINA ', 'MTTO TRANSPORTE', 'NO DEDUCIBLE', 'PAPELERIA',
       'PAQUETERIA', 'PEAJES', 'PUBLICIDAD', 'RENTA AUTO',
       'RENTA COPIADORA', 'SANITIZACION', 'SEGUROS', 'SUBCONTRATOS',
       'SUELDOS DIRECCION', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORMES', 'VALES DESPENSA', 'VARIOS'], "OTROS GASTOS")
gastos22

,Unnamed: 0,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,TC,Importe,IVA,RET ISR,RET IVA,Otros,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,0,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,1.0,5551.38,888.22,0.0,0.0,0.0,6439.60,6439.60,I,Vigente,PE02-ENE22
1,1,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,1.0,5551.98,888.32,0.0,0.0,0.0,6440.30,6440.30,I,Vigente,PE03-ENE22
2,2,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,Servicios de Facturación,PUE,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,I,Vigente,--
3,3,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,--,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
4,4,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,--,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,2572,2022-12-01,863376,E90DA089-3C3B-45C8-A19C-B8810D222F60,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",OTROS GASTOS,COMISIONES E INTERESES,PUE,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,I,Vigente,--
2573,2573,2022-12-01,1092957089,D720ED7C-F370-4236-B269-8873ABB78745,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,OTROS GASTOS,SERVICIOS BANCARIOS,PUE,1.0,17310.30,516.32,0.0,0.0,0.0,17826.62,17826.62,I,Vigente,--
2574,2574,2022-12-01,4,0AFD3EB5-F3DD-43EB-B9FA-79D793CFF417,AAE6411055P8,Aesa Articulos Electricos S.A de C.V,COMPRAS,Pago,--,--,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
2575,2575,2022-12-01,16,E3CA1DA8-581A-5844-80D2-CF0317B3BBEA,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,--,--,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--


Analizar los siguientes casos de correlación logística

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= gastos22[["TOTAL MX"]]
Var_Dep= gastos22["TIPO GASTO"]

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
      

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0 368]
 [  0 406]]


In [ ]:
#Calculo la precisión del modelo - OTROS GASTOS
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="OTROS GASTOS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.524547803617571


In [ ]:
#Calculo la precisión del modelo - COMPRAS
from sklearn.metrics import precision_score

precision2 = precision_score(y_test, y_pred, average="weighted", pos_label="COMPRAS")
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.27515039828001786


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'COMPRAS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.524547803617571


In [ ]:
#Calculo la sensibilidad del modelo - OTROS GASTOS
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="OTROS GASTOS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [ ]:
#Calculo la sensibilidad del modelo - COMPRAS
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y_test, y_pred, average="weighted", pos_label="COMPRAS")
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.524547803617571


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'COMPRAS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
